In [ ]:
pathes = [
    "./tmp/PianoFingeringDataset/FingeringFiles/001-1_fingering.txt",
    "./tmp/PianoFingeringDataset/FingeringFiles/001-2_fingering.txt",
    "./tmp/PianoFingeringDataset/FingeringFiles/001-5_fingering.txt",
    "./tmp/PianoFingeringDataset/FingeringFiles/001-8_fingering.txt",
    './data/001-my_fingering.csv',
    './data/001-Czerny_fingering.csv']
names = [getFilename(path) for path in pathes]
tables = [read_PIG(path) for path in pathes]
names

#fingersr = zip(*[table[table.ch == 0].finger for table in tables])
#fingersl = zip(*[table[table.ch == 1].finger for table in tables])

#score = makeScore(tables[0][tables[0].ch == 0], "test")
#score.write('musicxml', './mxl/bach_invention1.xml')

#score = mu.stream.Score()
#rights = tables[0][tables[0].ch == 0]
#appendPIGNotesWithMatches(rights, zip(rights.finger, czernyR)).write('lily.png')

In [ ]:
%load_ext autoreload
%autoreload 2

import music21 as mu
import pandas as pd
import numpy as np
from data_helper import *
from file_helper import *
from music_helper import *

def count_match(notes, fingers):
    """
    PIGデータと運指配列との一致数を返す
    """
    match = 0
    for i in range(len(notes)):
        if notes.iloc[i].finger == fingers.iat[i]:
            match += 1
    return match

def make_match_matrix(pig_array):
    """
    データ同士の運指の一致率のテーブルを返す
    """
    df = pd.DataFrame(columns = range(len(pig_array)))
    pig_fingers = [table.finger for table in pig_array]
    data_length = len(pig_fingers[0])
    for i in range(len(pig_fingers)):
        row = {}
        for j in range(len(pig_fingers)):
            fingerPair = zip(pig_fingers[i], pig_fingers[j]) # make pair
            cnt = 0
            for _, fs in enumerate(fingerPair):
                if len(set(fs)) < len(fs):
                    cnt += 1
            row[j] = cnt / data_length #rate = "{:.3f}".format(cnt / data_length)
        df = df.append(row, ignore_index = True)
    return df

def toColorCode(r, g, b):
    return "#{:02x}{:02x}{:02x}".format(r, g, b)

def noteWithFingering(note, pig_note, finger):
    note.articulations.append(mu.articulations.Fingering(finger))
    if pig_note.finger != finger:
        note.style.color = "red"
        note.addLyric(pig_note._finger)

def noteColoringByFingerMatches(note, fingers):
    differ = (len(set(fingers)) - 1) / (len(fingers) - 1)
    color = int((1 - pow(1 - differ, 2)) * 255)
    note.style.color = toColorCode(color, 0, 0)

def noteColoringByFingerJump(note1, note2):
    if note1 is None or note2 is None:
        return
    fd = note2.finger - note1.finger
    nd = note_dist(note1, note2)
    note2.articulations.append(mu.articulations.Fingering(fd))
    if abs(fd) > abs(nd):
        note.style.color = "red"
    
def appendPIGNotesWithFingerings(pig_notes, fingers):
    return appendPIGNotes(pig_notes, lambda i, n, c: noteWithFingering(n, c, fingers.iat[i]))

def appendPIGNotesWithMatches(pig_notes, fingerZipped):
    fingerlist = list(fingerZipped)
    return appendPIGNotes(pig_notes, lambda i, n, c: noteColoringByFingerMatches(n, fingerlist[i]))

def appendPIGNotesWithFingerJump(pig_notes):
    poses = np.arange(len(pig_notes))
    fingers = np.arange(len(pig_notes))
    stream = appendPIGNotes(pig_notes, lambda i, n, c: [poses.itemset(i, note_pos(n)), fingers.itemset(i, c.finger)])
    pi = 0
    for note in stream:
        if note.isRest:
            continue
        note.addLyric(fingers[pi])
        if pi > 0:
            pd = poses[pi] - poses[pi - 1]
            fd = fingers[pi] - fingers[pi - 1]
            note.articulations.append(mu.articulations.Fingering(pd))
            # 鍵盤上の距離より、指番号の差の方が大きい＝指寄せ
            if np.sign(pd) == np.sign(fd) and abs(pd) < abs(fd):
                note.style.color = "blue"
        pi += 1
    return stream

In [ ]:
df = make_match_matrix(tables)
print(df)

In [ ]:
for i in range(len(pathes)):
    score = appendPIGNotesWithFingerJump(tables[i][tables[i].ch == 0])
    output = f"./img/{names[i]}.xml"
    score.write("musicxml", output)
    score.write("musicxml.png", output)